In [ ]:
from dotenv import load_dotenv
load_dotenv()


True

In [8]:
# Example for Altering Human / AI messages.
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model_name= "gpt-3.5-turbo", temperature=0)

template="You are a helpful assistant that translates english to pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run("I love programming")

"I be lovin' programmin' arrr!"

In [9]:
from langchain import PromptTemplate, FewShotPromptTemplate
# Few shot prompting

examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(input_variables=['query', 'answer'], template=example_template)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

few_shot_prompt_template=FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix=suffix,
    prefix=prefix,
    input_variables=['query'],
)


In [10]:
chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the secret of happiness")

'The secret of happiness is a well-stocked fridge and a good Wi-Fi connection.'

In [ ]:
# length based fewshot prompting without semantic selector
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """
Word: {word}
Antonym: {antonym}
"""

In [13]:
example_prompt = PromptTemplate(
    input_variables=['word', 'antonym'],
    template=example_template
)

In [14]:
example_selector = LengthBasedExampleSelector(
    examples=example,
    example_prompt=example_prompt,
    max_length=25
)

In [15]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

In [16]:
print(dynamic_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad



Word: tall
Antonym: short



Word: energetic
Antonym: lethargic



Word: sunny
Antonym: gloomy


Word: big
Antonym:


# length based fewshot prompting with semantic selector

In [21]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Create promptTemplate 
example_prompt=PromptTemplate(
    input_variables=['input', 'output'],
    template="Input: {input}\nOutput: {output}",
)

# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

# Create Deep lake dataset
my_activeloop_ord_id = "charanvardhan"
my_activeloop_dataset_name = "langchain_fewshot_selector"
dataset_path  = f"hub://{my_activeloop_ord_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path)

# Embedding function
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

#Instantiate SemanticSimilarityExampleSelector using examples
example_selector = SemanticSimilarityExampleSelector.from_examples(examples, embeddings, db, k=1)

#Create a FewshotpromptTemplate using the example selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from celsius to fahrenheit",
    suffix="Input: {temperature}\nOutput:",
    input_variables=['temperature'],
)

# test the similar prompt with different inputs
print(similar_prompt.format(temperature="10°C"))   # Test with an input
print(similar_prompt.format(temperature="45°C"))

Your Deep Lake dataset has been successfully created!


Creating 5 embeddings in 1 batches of size 5:: 100%|██████████| 1/1 [00:01<00:00,  1.15s/it]


Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (5, 1)      str     None   
 metadata     json      (5, 1)      str     None   
 embedding  embedding  (5, 1536)  float32   None   
    id        text      (5, 1)      str     None   
Convert the temperature from celsius to fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from celsius to fahrenheit

Input: 40°C
Output: 104°F

Input: 45°C
Output:


In [22]:
# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="55C")) # Test with a new input afer adding the example

Creating 1 embeddings in 1 batches of size 1:: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s]


Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   
Convert the temperature from celsius to fahrenheit

Input: 50°C
Output: 122°F

Input: 55C
Output:
